In [ ]:
!pip install -q transformers datasets accelerate
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "facebook/esm2_t6_8M_UR50D"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset, ClassLabel

# 1. Load raw CSV into HF Dataset
df = pd.read_csv("/content/drive/MyDrive/AllergenAInew/ESMFold/algpred2_train.csv")
dataset = Dataset.from_pandas(df)

# 2. Convert 'label' to ClassLabel BEFORE tokenization
dataset = dataset.cast_column("label", ClassLabel(num_classes=2))

# 3. Tokenize sequences
def tokenize(example):
    return tokenizer(example["sequence"], padding="max_length", truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=True)

# 4. Now do stratified split
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, stratify_by_column="label")


Casting the dataset:   0%|          | 0/16120 [00:00<?, ? examples/s]

Map:   0%|          | 0/16120 [00:00<?, ? examples/s]

Tokenize the sequences

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

accuracy = evaluate.load("accuracy")
roc_auc = evaluate.load("roc_auc", "binary")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    probas = tf.nn.softmax(logits, axis=-1).numpy()
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "roc_auc": roc_auc.compute(prediction_scores=probas[:, 1], references=labels)["roc_auc"]
    }

training_args = TrainingArguments(
    output_dir="./esm2_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-bfb0a2cb9544>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 6. Train the Model

In [ ]:
# ==============================
# 📈 Metrics: Accuracy, ROC-AUC, F1
# ==============================
accuracy = evaluate.load("accuracy")
roc_auc = evaluate.load("roc_auc", "binary")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    probas = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "roc_auc": roc_auc.compute(prediction_scores=probas[:, 1], references=labels)["roc_auc"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }

# ==============================
# ⚙️ Training Arguments
# ==============================
training_args = TrainingArguments(
    output_dir="./esm2_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    run_name="esm2_finetuned"
)

# ==============================
# 🧠 Trainer
# ==============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# ==============================
# 🏋️‍♀️ Fine-tune!
# ==============================
trainer.train()

# ==============================
# 📊 Final Evaluation with SE
# ==============================
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.utils import resample
import numpy as np

# Get predictions
predictions = trainer.predict(tokenized_dataset["test"])
logits = predictions.predictions
labels = predictions.label_ids

# Softmax for probabilities
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
y_pred = np.argmax(probs, axis=1)
y_true = labels
y_scores = probs[:, 1]

# Bootstrap SE function
def bootstrap_metric(metric_fn, y_true, y_score, n_rounds=1000):
    scores = []
    for _ in range(n_rounds):
        idx = np.random.choice(len(y_true), len(y_true), replace=True)
        score = metric_fn(y_true[idx], y_score[idx])
        scores.append(score)
    return np.mean(scores), np.std(scores, ddof=1) / np.sqrt(n_rounds)

# Compute metrics + SE
acc_mean, acc_se = bootstrap_metric(accuracy_score, y_true, y_pred)
auc_mean, auc_se = bootstrap_metric(roc_auc_score, y_true, y_scores)
f1_mean, f1_se = bootstrap_metric(f1_score, y_true, y_pred)

# Print results
print("\n✅ Final Metrics on Test Set (with Standard Error):")
print(f"Accuracy:  {acc_mean:.4f} ± {acc_se:.4f} (SE)")
print(f"ROC-AUC:   {auc_mean:.4f} ± {auc_se:.4f} (SE)")
print(f"F1 Score:  {f1_mean:.4f} ± {f1_se:.4f} (SE)")

# ==============================
# 💾 Save Best Model
# ==============================
trainer.save_model("/content/drive/MyDrive/AllergenAInew/Fine-tune Transformer/esm2_finetuned_best")
tokenizer.save_pretrained("/content/drive/MyDrive/AllergenAInew/Fine-tune Transformer/esm2_finetuned_best")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jefferygermany (jefferygermany-eth-z-rich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

🧪 7. Evaluate on Test Set

In [ ]:
metrics = trainer.evaluate()
print(metrics)

🧬 Optional: Scale Up to Bigger ESM-2
Once it works, you can change:

In [ ]:
model_name = "facebook/esm2_t33_650M_UR50D"  # or t36_3B if you dare